In [1]:
import sklearn.metrics
import autosklearn.regression
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import talib

In [2]:
# Get the data for the SPY ETF by specifying the stock ticker, start date, and end date
data = yf.download('0005.hk','2015-01-01','2020-01-01')

[*********************100%***********************]  1 of 1 completed


In [3]:
data['H-L'] = data['High'] - data['Low']
data['O-C'] = data['Close'] - data['Open']
data['3day MA'] = data['Close'].shift(1).rolling(window = 3).mean()
data['10day MA'] = data['Close'].shift(1).rolling(window = 10).mean()
data['30day MA'] = data['Close'].shift(1).rolling(window = 30).mean()
data['Std_dev']= data['Close'].rolling(5).std()
data['RSI'] = talib.RSI(data['Close'].values, timeperiod = 9)
data['Williams %R'] = talib.WILLR(data['High'].values, data['Low'].values, data['Close'].values, 7)
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'H-L', 'O-C',
       '3day MA', '10day MA', '30day MA', 'Std_dev', 'RSI', 'Williams %R'],
      dtype='object')

In [4]:
data.dropna(inplace=True)

In [5]:
train = data[0:int(len(data)*0.99)]
data_price=train["Close"]
train = train.values
# scl = MinMaxScaler()
# #Scale the data
# scl.fit(train.values.reshape(-1,1))
# cl =scl.transform(cl.values.reshape(-1,1))
# sc = MinMaxScaler(feature_range=(0,1))
# train=sc.fit_transform(train)
def processData(data,data_price,lb):
    X,Y = [],[]
    for i in range(len(data)-lb-1):
        X.append(data[i:(i+lb)])
        Y.append(data_price[(i+lb)])
    return np.array(X),np.array(Y)

lb=3
X,y = processData(train,data_price,lb)
# X_train,X_test = X[:int(X.shape[0]*0.90)],X[int(X.shape[0]*0.90):]
# y_train,y_test = y[:int(y.shape[0]*0.90)],y[int(y.shape[0]*0.90):]
# print(X_train.shape[0],X_train.shape[1])
# print(X_test.shape[0], X_test.shape[1])
# print(y_train.shape[0])
# print(y_test.shape[0])

In [6]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42)
print(X_train.shape[0],X_train.shape[1])
print(X_test.shape[0], X_test.shape[1])
print(y_train.shape[0])
print(y_test.shape[0])

889 3
297 3
889
297


In [7]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    memory_limit=5000,
)
automl.fit(X_train, y_train)

Process pynisher function call:
Traceback (most recent call last):
  File "/home/gordon/anaconda3/envs/FYP/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/gordon/anaconda3/envs/FYP/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gordon/anaconda3/envs/FYP/lib/python3.8/site-packages/pynisher/limit_function_call.py", line 133, in subprocess_func
    return_value = ((func(*args, **kwargs), 0))
  File "/home/gordon/anaconda3/envs/FYP/lib/python3.8/site-packages/autosklearn/smbo.py", line 99, in _calculate_metafeatures
    result = calculate_all_metafeatures_with_labels(
  File "/home/gordon/anaconda3/envs/FYP/lib/python3.8/site-packages/autosklearn/metalearning/metafeatures/metafeatures.py", line 1051, in calculate_all_metafeatures_with_labels
    return calculate_all_metafeatures(X, y, categorical, dataset_name,
  File "/home/gordon/anaconda3/envs/FYP/lib/python3.8/site-

AutoSklearnRegressor(memory_limit=5000, per_run_time_limit=30,
                     time_left_for_this_task=120)

In [8]:
print(automl.leaderboard())

KeyError: 1

In [ ]:
print(automl.show_models())

In [ ]:
train_predictions = automl.predict(X_train)
print("Train R2 score:", sklearn.metrics.r2_score(y_train, train_predictions))
test_predictions = automl.predict(X_test)
print("Test R2 score:", sklearn.metrics.r2_score(y_test, test_predictions))

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.set_facecolor('#000041')
ax.plot(y_train, color='red', label='Original price')
plt.plot(train_predictions, color='cyan', label='Predicted price')
plt.legend()

In [ ]:
data_val=yf.download('0005.hk','2020-01-02','2020-01-31')
x_val=np.array(np.array(data_val.drop(["Adj Close","Volume"],1)))
y_val=np.array(data_val['Adj Close'])
predictions = automl.predict(x_val)
# predictions = scaler.inverse_transform(predictions.reshape(-1,1))
# y_test_scaled = scaler.inverse_transform(y_test.reshape(-1, 1))

fig, ax = plt.subplots(figsize=(16,8))
ax.set_facecolor('#000041')
ax.plot(y_val, color='red', label='Original price')
plt.plot(predictions, color='cyan', label='Predicted price')
plt.legend()

In [ ]:
test=make_pipeline(StandardScaler(),autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    memory_limit=5000,
))
test.fit(X_train, y_train)

In [ ]:
train_predictions = test.predict(X_train)
print("Train R2 score:", sklearn.metrics.r2_score(y_train, train_predictions))
test_predictions = test.predict(X_test)
print("Test R2 score:", sklearn.metrics.r2_score(y_test, test_predictions))

In [ ]:
data_val=yf.download('0005.hk','2020-01-02','2020-01-31')
x_val=np.array(np.array(data_val.drop(["Adj Close","Volume"],1)))
y_val=np.array(data_val['Adj Close'])
predictions = automl.predict(x_val)
# predictions = scaler.inverse_transform(predictions.reshape(-1,1))
# y_test_scaled = scaler.inverse_transform(y_test.reshape(-1, 1))

fig, ax = plt.subplots(figsize=(16,8))
ax.set_facecolor('#000041')
ax.plot(y_val, color='red', label='Original price')
plt.plot(predictions, color='cyan', label='Predicted price')
plt.legend()